In [53]:
# to detect and crop faces and save it in folders 

import os
import numpy as np
from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, Colors
from pathlib import Path

def plot_bounding_boxes(yolo_result, colors=Colors()):
    """Draw bounding boxes from yolo.predict() result"""
    image = yolo_result.orig_img[..., ::-1]
    annotate = Annotator(np.ascontiguousarray(image))
    classes = yolo_result.boxes.cls.tolist()
    scores = yolo_result.boxes.conf.tolist()
    boxes = yolo_result.boxes.xyxy
    for box, class_, score in zip(boxes, classes, scores):
        color = np.random.randint(0, 255, 3).tolist()
        tag = f"{yolo_result.names[class_].title()}: {score:.0%}"
        annotate.box_label(box, tag, color)
    return Image.fromarray(annotate.result())

def crop_faces(yolo_result, image_path, output_path):
    """Crop faces from the image based on YOLO result and save them"""
    try:
        image = Image.open(image_path).convert('RGB')  # Convert to RGB to avoid RGBA issue
    except FileNotFoundError:
        print(f"Image Not Found: {image_path}")
        return

    boxes = yolo_result.boxes.xyxy
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        face = image.crop((x1, y1, x2, y2))
        face_path = os.path.join(output_path, f"{Path(image_path).stem}_face_{i}.jpg")
        face.save(face_path)

# Define paths
data_dir = '/mnt/c/Users/user/Desktop/therd_try/persons'
output_dir = 'cropped_faces_no_labeled_players'

# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLO model
model = YOLO('yolov8n_face.pt')

# Process each image in the dataset
for person_folder in os.listdir(data_dir):
    person_path = os.path.join(data_dir, person_folder)
    if os.path.isdir(person_path):
        person_output_path = os.path.join(output_dir, person_folder)
        os.makedirs(person_output_path, exist_ok=True)

        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            if not os.path.isfile(image_path):
                print(f"Image Not Found: {image_path}")
                continue

            result = model(image_path)[0]
            crop_faces(result, image_path, person_output_path)

print("Face detection and cropping completed.")


image 1/1 /mnt/c/Users/user/Desktop/therd_try/persons/elton/10_6l7wZPW.jpg: 416x640 1 face, 80.1ms
Speed: 7.5ms preprocess, 80.1ms inference, 1598.9ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /mnt/c/Users/user/Desktop/therd_try/persons/elton/1634532.jpg: 448x640 1 face, 66.4ms
Speed: 5.0ms preprocess, 66.4ms inference, 2.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /mnt/c/Users/user/Desktop/therd_try/persons/elton/2013_8_14_2_13_10_833.jpg: 384x640 3 faces, 133.3ms
Speed: 4.3ms preprocess, 133.3ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /mnt/c/Users/user/Desktop/therd_try/persons/elton/2015-09-04_20-38-28.jpg: 640x448 2 faces, 57.2ms
Speed: 4.3ms preprocess, 57.2ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 /mnt/c/Users/user/Desktop/therd_try/persons/elton/2102658.jpg: 640x480 1 face, 58.5ms
Speed: 5.9ms preprocess, 58.5ms inference, 2.4ms postprocess per image at shape (1, 3, 640,

In [2]:
import keras
from keras import ops
from keras import layers
from keras.models import load_model


# Distorts the color distibutions of images
class RandomColorAffine(layers.Layer):
    def __init__(self, brightness=0, jitter=0, **kwargs):
        super().__init__(**kwargs)

        self.seed_generator = keras.random.SeedGenerator(1337)
        self.brightness = brightness
        self.jitter = jitter

    def get_config(self):
        config = super().get_config()
        config.update({"brightness": self.brightness, "jitter": self.jitter})
        return config

    def call(self, images, training=True):
        if training:
            batch_size = ops.shape(images)[0]

            # Same for all colors
            brightness_scales = 1 + keras.random.uniform(
                (batch_size, 1, 1, 1),
                minval=-self.brightness,
                maxval=self.brightness,
                seed=self.seed_generator,
            )
            # Different for all colors
            jitter_matrices = keras.random.uniform(
                (batch_size, 1, 3, 3), 
                minval=-self.jitter, 
                maxval=self.jitter,
                seed=self.seed_generator,
            )

            color_transforms = (
                ops.tile(ops.expand_dims(ops.eye(3), axis=0), (batch_size, 1, 1, 1))
                * brightness_scales
                + jitter_matrices
            )
            images = ops.clip(ops.matmul(images, color_transforms), 0, 1)
        return images


labeled_train_dataset, test_dataset = keras.utils.image_dataset_from_directory(
    "/mnt/c/Users/user/Desktop/therd_try/cropped_faces_no_labeled_players",
    color_mode="rgb",
    image_size=(128, 128),
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    interpolation='bilinear',
    seed=123,
    shuffle=True,
    validation_split=0.2,
    subset='both',
    batch_size=8,
)

labeled_train_dataset.class_names

2024-05-27 16:07:26.298227: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 16:07:27.729856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 106 files belonging to 3 classes.
Using 85 files for training.
Using 21 files for validation.


2024-05-27 16:07:30.359613: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 16:07:30.909343: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 16:07:30.909408: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 16:07:30.912479: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 16:07:30.912557: I external/local_xla/xla/stream_executor

['elton', 'malk', 'yasser']

In [3]:
keras_model = keras.models.Sequential(
    [
        keras.models.load_model('./keras_model_classify.keras').backbone,
        keras.layers.GlobalAveragePooling2D(),
    ]
)
baseline_model = keras.models.load_model('./baseline_model.keras', custom_objects={'RandomColorAffine': RandomColorAffine})
# pretraining_model = keras.models.load_model('./pretraining_model.keras', custom_objects={'RandomColorAffine': RandomColorAffine})
finetuning_model = keras.models.load_model('./finetuning_model.keras', custom_objects={'RandomColorAffine':RandomColorAffine})

baseline_model.pop()
finetuning_model.pop()

<Dense name=dense_3, built=True>

In [65]:
import numpy as np
import cv2

def preprocess(image_path):
    img = cv2.imread(image_path)[..., ::-1]
    img_resize = cv2.resize(img, (128, 128))
    img = np.expand_dims(img_resize, axis=0)
    return img

def get_features(model, image):
    if isinstance(image, str):
        image = preprocess(image)
    return model(image, training=True).numpy()[0]

def distance(x, y):
    # Eucleadian Distance
    return (((x - y)**2).sum())**0.5

def similarity(x, y):
    # Cosine Similarity
    x = x / np.linalg.norm(x)
    y = y / np.linalg.norm(y)
    return (x * y).sum()

In [32]:
model = finetuning_model

unknown = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/persons/elton/99-140348-elton-jose-wahda-ksa_700x400.png')

elton = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/persons/elton/2102658.jpg')
malk = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/persons/malk/6432.jpg')
yasser = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/persons/yasser/1ea60361-41d2-4791-a79e-74df9d2dd20a.jpg')

print('Similarity')
print('elton', similarity(unknown, elton))
print('malk', similarity(unknown, malk))
print('yasser', similarity(unknown, yasser))
print()
print('elton', distance(unknown, elton))
print('malk', distance(unknown, malk))
print('yasser', distance(unknown, yasser))

Similarity
elton 0.087997876
malk 0.3689572
yasser 0.26021373

elton 34.4320141530061
malk 24.98391694584801
yasser 29.096926014883856


In [69]:
model = baseline_model

unknown = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/cropped_faces_no_labeled_players/elton/8AFD98A-D98AD981D8A7D988D8B6-D8A3D984D8AAD988D986-D8ACD988D8B2D98AD987-49b15_face_0.jpg')

elton = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/cropped_faces_no_labeled_players/elton/99-140348-elton-jose-wahda-ksa_700x400_face_0.jpg')
malk = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/cropped_faces_no_labeled_players/malk/7D984D8A7D8AAD8ADD8A7D8AF-D8A8D8AFD988D986-D985D982D8A7D8A8D984-D984D8B1D8AF_face_0.jpg')
yasser = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/cropped_faces_no_labeled_players/yasser/1ea60361-41d2-4791-a79e-74df9d2dd20a_face_0.jpg')

print('Similarity')
print('elton', similarity(unknown, elton))
print('malk', similarity(unknown, malk))
print('yasser', similarity(unknown, yasser))
print()
print('elton', distance(unknown, elton))
print('malk', distance(unknown, malk))
print('yasser', distance(unknown, yasser))

Similarity
elton 0.5474911
malk 0.73349345
yasser 0.53449214

elton 1.7033311911542726
malk 1.2599505148358823
yasser 1.952624569678853


In [27]:
model = finetuning_model

elton1 = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/persons/malk/43c41ea5-f913-4c3d-8a4f-aa05c686924b.jpeg')
elton1.max()

5.0031505

In [28]:
elton2 = get_features(model, '/mnt/c/Users/user/Desktop/therd_try/persons/malk/43c41ea5-f913-4c3d-8a4f-aa05c686924b.jpeg')
elton2.max()

5.0031505

In [29]:
print('elton', similarity(elton1, elton2))
print('elton', distance(elton1, elton2))

elton 1.0
elton 0.0
